In [1]:
!pip install yfinance plotly

In [2]:
import yfinance as yf
import pandas as pd
import plotly.graph_objs as go
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np


In [3]:
# Function to fetch data
def get_stock_data(ticker, period='1d', interval='1m'):
    stock = yf.Ticker(ticker)
    data = stock.history(period=period, interval=interval)
    return data


In [4]:
data = get_stock_data('AAPL', period='1d', interval='1m')
data = data.dropna()
data['Timestamp'] = data.index.astype(int) / 10**9
X = data['Timestamp'].values.reshape(-1, 1)
y = data['Close'].values


In [5]:
print("Last data timestamp:", data.index[-1])


Last data timestamp: 2025-06-23 11:36:00-04:00


In [8]:
# List of stocks to compare
tickers = ['AAPL', 'MSFT', 'GOOGL']

# Fetch and store all data
all_data = {}
for ticker in tickers:
    all_data[ticker] = get_stock_data(ticker)

# Plotting function
def plot_multiple_stocks(all_data, tickers):
    fig = go.Figure()
    for ticker in tickers:
        fig.add_trace(go.Scatter(
            x=all_data[ticker].index,
            y=all_data[ticker]['Close'],
            mode='lines',
            name=f'{ticker} Close Price'
        ))

    fig.update_layout(
        title='Real-Time Stock Prices Comparison',
        xaxis_title='Time',
        yaxis_title='Price (USD)',
        template='plotly_dark',
        width=1000,
        height=500
    )
    fig.show()

# Plot multiple stocks
plot_multiple_stocks(all_data, tickers)


In [9]:
from plotly.subplots import make_subplots

data = yf.Ticker('AAPL').history(period='5d', interval='1h')

fig = make_subplots(rows=2, cols=1, shared_xaxes=True,
                    row_heights=[0.7, 0.3],
                    subplot_titles=('Price', 'Volume'))

fig.add_trace(go.Scatter(
    x=data.index, y=data['Close'], name='Close Price'), row=1, col=1)

fig.add_trace(go.Bar(
    x=data.index, y=data['Volume'], name='Volume'), row=2, col=1)

fig.update_layout(
    title='AAPL Price and Volume Comparison',
    template='plotly_dark',
    width=1000,
    height=600
)
fig.show()


In [15]:
data['MA20'] = data['Close'].rolling(window=20).mean()
data['MA50'] = data['Close'].rolling(window=50).mean()

fig = go.Figure()
fig.add_trace(go.Scatter(x=data.index, y=data['Close'], name='Close Price'))
fig.add_trace(go.Scatter(x=data.index, y=data['MA20'], name='MA20'))
fig.add_trace(go.Scatter(x=data.index, y=data['MA50'], name='MA50'))

fig.update_layout(title='AAPL with Moving Averages', template='plotly_dark')
fig.show()


In [11]:
# Function to plot candlestick chart
def plot_candlestick(ticker, period='5d', interval='15m'):
    data = get_stock_data(ticker, period, interval)

    fig = go.Figure(data=[go.Candlestick(
        x=data.index,
        open=data['Open'],
        high=data['High'],
        low=data['Low'],
        close=data['Close']
    )])

    fig.update_layout(
        title=f'{ticker} Candlestick Chart',
        xaxis_title='Time',
        yaxis_title='Price (USD)',
        template='plotly_dark',
        width=1000,
        height=600
    )

    fig.show()

# Example usage
plot_candlestick('AAPL', period='5d', interval='15m')


In [12]:
# Function to calculate accuracy and plot actual vs predicted prices
def calculate_accuracy_and_plot(ticker, data):
    # Clean and prepare the data
    data = data.dropna()
    data['Timestamp'] = data.index.astype(int) / 10**9

    X = data['Timestamp'].values.reshape(-1, 1)
    y = data['Close'].values

    # Linear Regression Model
    model = LinearRegression()
    model.fit(X, y)
    y_pred = model.predict(X)

    # Calculate MSE and R²
    mse = mean_squared_error(y, y_pred)
    r2 = r2_score(y, y_pred)

    print(f'\n📊 Accuracy for {ticker}:')
    print(f'Mean Squared Error (MSE): {mse:.2f}')
    print(f'R² Score: {r2:.4f}')

    # Plot Actual vs Predicted
    fig = go.Figure()

    fig.add_trace(go.Scatter(
        x=data.index, y=y, mode='lines', name='Actual Price'))

    fig.add_trace(go.Scatter(
        x=data.index, y=y_pred, mode='lines', name='Predicted Price'))

    fig.update_layout(
        title=f'{ticker} Actual vs Predicted Prices',
        xaxis_title='Time',
        yaxis_title='Price (USD)',
        template='plotly_dark',
        width=1000,
        height=500
    )

    fig.show()


In [13]:
# Run accuracy calculation and plotting for each stock
for ticker in tickers:
    calculate_accuracy_and_plot(ticker, all_data[ticker])



📊 Accuracy for AAPL:
Mean Squared Error (MSE): 0.18
R² Score: 0.3671



📊 Accuracy for MSFT:
Mean Squared Error (MSE): 0.37
R² Score: 0.8402



📊 Accuracy for GOOGL:
Mean Squared Error (MSE): 0.37
R² Score: 0.2217


In [14]:
# Function to download CSV
def download_historical_data(ticker, period='1y', interval='1d'):
    data = get_stock_data(ticker, period, interval)
    filename = f'{ticker}_historical_data.csv'
    data.to_csv(filename)
    return filename

# Example download
file_name = download_historical_data('AAPL', period='1y', interval='1d')
print(f'Downloaded historical data to {file_name}')


Downloaded historical data to AAPL_historical_data.csv
